In [1]:
# Install Rasa
!pip install rasa

# Create Rasa Project
!rasa init

# Modify 'nlu.yml' to include intents and examples for retrieval-based queries
# Modify 'stories.yml' to define conversational flows
# Modify 'domain.yml' to define entities, slots, and responses
# Modify 'actions.py' to implement custom actions using TF-IDF and rule-based logic

# Train Rasa Model
!rasa train

# Run Rasa Server
!rasa run

# Run Rasa Action Server
!rasa run actions


  Obtaining dependency information for rasa from https://files.pythonhosted.org/packages/e7/95/b5e9380fb5e64d478f098f02bcf4014cd74bf5dd24b26687d0c0bc08f43f/rasa-1.10.2-py3-none-any.whl.metadata
  Obtaining dependency information for PyJWT<1.8,>=1.7 from https://files.pythonhosted.org/packages/87/8b/6a9f14b5f781697e51259d81657e6048fd31a113229cf346880bb7545565/PyJWT-1.7.1-py2.py3-none-any.whl.metadata
     ---------------------------------------- 0.0/6.4 MB ? eta -:--:--
     ---------------------------------------- 0.0/6.4 MB 1.4 MB/s eta 0:00:05
      --------------------------------------- 0.1/6.4 MB 1.2 MB/s eta 0:00:06
     ------- -------------------------------- 1.2/6.4 MB 9.9 MB/s eta 0:00:01
     --------------------------- ------------ 4.3/6.4 MB 27.5 MB/s eta 0:00:01
     ---------------------------------------  6.3/6.4 MB 33.6 MB/s eta 0:00:01
     ---------------------------------------  6.3/6.4 MB 33.6 MB/s eta 0:00:01
     ---------------------------------------  6.3/6.4 M

  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [6 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\HUAWEI\AppData\Local\Temp\pip-install-y4hi47mm\absl-py_4662ad691ef145e68d8fdc35b576bf26\setup.py", line 34, in <module>
      raise RuntimeError('Python version 2.7 or 3.4+ is required.')
  RuntimeError: Python version 2.7 or 3.4+ is required.
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
'rasa' is not recognized as an internal or external command,
operable program or batch file.
'rasa' is not recognized as an internal or external command,

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Sample data
data = [
    {"program": "Computer Science", "university": "University A", "fees": 20000, "duration": "4 years", "location": "City X"},
    {"program": "Data Science", "university": "University B", "fees": 25000, "duration": "2 years", "location": "City Y"},
    # Add more entries
]

file_path = r'C:\Users\HUAWEI\AIAssignment\Food Ingredients and Recipe Dataset with Image Name Mapping.csv'

df = pd.DataFrame(data)

# User query
user_query = "computer science university fees"

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['program'] + ' ' + df['university'] + ' ' + df['location'])

# Transform user query
query_vec = vectorizer.transform([user_query])

# Compute similarity
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(query_vec, tfidf_matrix).flatten()

# Get top match
top_match_idx = similarities.argmax()
top_match = df.iloc[top_match_idx]

print("Matched Program/University:", top_match)


Matched Program/University: program       Computer Science
university        University A
fees                     20000
duration               4 years
location                City X
Name: 0, dtype: object


In [3]:
pip install pyodbc pandas

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pyodbc
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Retrieve data from the database
query = "SELECT * FROM Programme"
cursor.execute(query)
rows = cursor.fetchall()

# Get column names
columns = [column[0] for column in cursor.description]

# Convert data to a DataFrame
df = pd.DataFrame.from_records(rows, columns=columns)

# Close the connection
cursor.close()
conn.close()

# Ensure all relevant columns are strings and handle missing values
df['programName'] = df['programName'].astype(str).fillna('')
df['introduction'] = df['introduction'].astype(str).fillna('')
df['duration'] = df['duration'].astype(str).fillna('')
df['fees'] = df['fees'].astype(str).fillna('')

# Combine text fields into a single column
df['combined'] = df['programName'] + ' ' + df['introduction'] + ' ' + df['duration'] + ' ' + df['fees']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined'])

print("EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?")
print()

# Function to calculate cosine similarity between a user input vector and database vectors
def calculate_similarity(query_vec, tfidf_matrix):
    return cosine_similarity(query_vec, tfidf_matrix)

# Loop for interactive user input and recommendation
while True:
    # Get user input
    user_input = input("> What is your programme criteria? ")
    
    # Exit loop if user enters 'exit'
    if user_input.lower() == 'exit':
        print("EduBot: I am happy to be at your service. See you again!")
        break
    
    # Generate the user input vector using the TF-IDF model
    query_vec = vectorizer.transform([user_input])
    
    # Calculate similarity between user input vector and database vectors
    similarities = calculate_similarity(query_vec, tfidf_matrix)

    # Get the indices of programmes sorted by similarity (in descending order)
    top_indices = np.argsort(similarities[0])[::-1]
    
    print()
    print("EduBot: Here are the results of programme recommendations based on your criteria.")
    print()
    
    print("Top 5 Programme Recommendations:")
    print()
    
    counter = 0
 
    # Display the top 5 programme recommendations
    for counter, index in enumerate(top_indices[:5], start=1):
        print()
        print(f"{counter}. Program: {df['programName'][index]}")

        print("Introduction:")
        print(df['introduction'][index])
        print()
        
        print("Duration:")
        print(df['duration'][index])
        print()
        
        print("Fees:")
        print(df['fees'][index])
        print()
        
    print("Enter another set of programme criteria to generate other results, or type 'exit' to end the session...")


EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?

> What is your programme criteria? arts animation 30000

EduBot: Here are the results of programme recommendations based on your criteria.

Top 5 Programme Recommendations:


1. Program: Bachelor of Design (Animation) with Honours

Introduction:
The university has not updated this information during the data collection process.<br/>Please refer to the latest updates on the university's website.

Duration:
4

Fees:
13344


2. Program: Bachelor of Education (Arts) with Honours

Introduction:
The university has not updated this information during the data collection process.<br/>Please refer to the latest updates on the university's website.

Duration:
4

Fees:
11736


3. Program: Bachelor of Performing Arts (Theatre) with Honours

Introduction:
The university has not updated this information during the data collection process.<br/>Please refer to the latest updates on the universit

In [ ]:
#version 2

import pyodbc
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Retrieve data from the database
query = "SELECT * FROM Programme"
cursor.execute(query)
rows = cursor.fetchall()

# Get column names
columns = [column[0] for column in cursor.description]

# Convert data to a DataFrame
df = pd.DataFrame.from_records(rows, columns=columns)

# Close the connection
cursor.close()
conn.close()

# Ensure all relevant columns are strings and handle missing values
df['programName'] = df['programName'].astype(str).fillna('')
df['introduction'] = df['introduction'].astype(str).fillna('')
df['duration'] = df['duration'].astype(str).fillna('')
df['fees'] = pd.to_numeric(df['fees'], errors='coerce').fillna(0)  # Ensure fees are numeric

# Combine text fields into a single column for TF-IDF
df['combined'] = df['programName'] + ' ' + df['introduction'] + ' ' + df['duration']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined'])

print("EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?")
print()

def calculate_similarity(query_vec, tfidf_matrix):
    return cosine_similarity(query_vec, tfidf_matrix)

while True:
    # Get user input
    user_input = input("> What is your programme criteria? ")
    
    # Exit loop if user enters 'exit'
    if user_input.lower() == 'exit':
        print("EduBot: I am happy to be at your service. See you again!")
        break
    
    # Generate the user input vector using the TF-IDF model
    query_vec = vectorizer.transform([user_input])
    
    # Calculate similarity between user input vector and database vectors
    similarities = calculate_similarity(query_vec, tfidf_matrix)

    # Get the indices of programmes sorted by similarity (in descending order)
    top_indices = np.argsort(similarities[0])[::-1]
    
    # Get top 10 results to apply numerical filtering
    top_results = df.iloc[top_indices[:10]]

    print()
    print("EduBot: Here are the results of programme recommendations based on your criteria.")
    print()
    
    print("Top 5 Programme Recommendations:")
    print()
    
    # Example numerical filters (modify as needed)
    max_fees = float(input("> Enter maximum fees: "))
    max_duration = int(input("> Enter maximum duration in years: "))

    # Apply numerical filters
    filtered_results = top_results[(top_results['fees'] <= max_fees) & (top_results['duration'].astype(int) <= max_duration)]
    
    # Display the top 5 filtered results
    for counter, index in enumerate(filtered_results.head(5).index, start=1):
        print()
        print(f"{counter}. Program: {df['programName'][index]}")
        print("Introduction:")
        print(df['introduction'][index])
        print()
        print("Duration:")
        print(df['duration'][index])
        print()
        print("Fees:")
        print(df['fees'][index])
        print()
        
    print("Enter another set of programme criteria to generate other results, or type 'exit' to end the session...")


In [18]:
#version 3
import pyodbc
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Retrieve data from the database
query = "SELECT * FROM Programme"
cursor.execute(query)
rows = cursor.fetchall()

# Get column names
columns = [column[0] for column in cursor.description]

# Convert data to a DataFrame
df = pd.DataFrame.from_records(rows, columns=columns)

# Close the connection
cursor.close()
conn.close()

# Ensure all relevant columns are strings and handle missing values
df['programName'] = df['programName'].astype(str).fillna('')
df['introduction'] = df['introduction'].astype(str).fillna('')
df['duration'] = df['duration'].astype(str).fillna('')
df['fees'] = pd.to_numeric(df['fees'], errors='coerce').fillna(0)  # Ensure fees are numeric

# Combine text fields into a single column for TF-IDF
df['combined'] = df['programName'] + ' ' + df['introduction'] + ' ' + df['duration']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined'])

print("EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?")
print()

def calculate_similarity(query_vec, tfidf_matrix):
    return cosine_similarity(query_vec, tfidf_matrix)

while True:
    # Get user input
    user_input = input("> What is your programme criteria? ")
    
    # Exit loop if user enters 'exit'
    if user_input.lower() == 'exit':
        print("EduBot: I am happy to be at your service. See you again!")
        break
    
    # Generate the user input vector using the TF-IDF model
    query_vec = vectorizer.transform([user_input])
    
    # Calculate similarity between user input vector and database vectors
    similarities = calculate_similarity(query_vec, tfidf_matrix)

    # Get the indices of programmes sorted by similarity (in descending order)
    top_indices = np.argsort(similarities[0])[::-1]
    
    # Get top 10 results to apply numerical filtering
    top_results = df.iloc[top_indices[:10]]

    if top_results.empty:
        print("EduBot: No programs found based on the text criteria.")
        print("Please adjust your criteria or try a different query.")
        continue
    
    # Example numerical filters (modify as needed)
    max_fees = float(input("> Enter maximum fees: "))
    max_duration = int(input("> Enter maximum duration in years: "))

    # Apply numerical filters
    filtered_results = top_results[(top_results['fees'] <= max_fees) & (top_results['duration'].astype(int) <= max_duration)]
    
    if filtered_results.empty:
        print("EduBot: No programs found matching your numerical criteria.")
        print("Try adjusting the maximum fees and duration.")
        continue
    
    # Display the top 5 filtered results
    print("Top 5 Programme Recommendations:")
    print()
    
    for counter, index in enumerate(filtered_results.head(5).index, start=1):
        print()
        print(f"{counter}. Program: {df['programName'][index]}")
        print("Introduction:")
        print(df['introduction'][index])
        print()
        print("Duration:")
        print(df['duration'][index])
        print()
        print("Fees:")
        print(df['fees'][index])
        print()
        
    print("Enter another set of programme criteria to generate other results, or type 'exit' to end the session...")


EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?

> What is your programme criteria? business
> Enter maximum fees: 20000
> Enter maximum duration in years: 3
Top 5 Programme Recommendations:


1. Program: BACHELOR OF BUSINESS (HONOURS) (INTERNATIONAL BUSINESS) 
Introduction:
The Bachelor of International Business [Hons] programme was established to produce students who are knowledgeable and able to conduct business in the international arena. Thus, the root of knowledge in business and management disciplines is essential preparation for facing a more competitive in the business world whether in private or public organizations particularly the multinational corporations, non-governmental organizations (NGO?s), small-and-medium enterprises (SMEs) and other business entities and business service providers.

Duration:
3

Fees:
14245

Enter another set of programme criteria to generate other results, or type 'exit' to end the sessio

> What is your programme criteria? culinary
> Enter maximum fees: 40000
> Enter maximum duration in years: 4
Top 5 Programme Recommendations:


1. Program: Bachelor of Science with Education (Mathematics) with Honours
Introduction:
The university has not updated this information during the data collection process.<br/>Please refer to the latest updates on the university's website.

Duration:
4

Fees:
11960


2. Program: Bachelor of Chemical Process Engineering with Honours
Introduction:
5The Chemical Process Engineering program is offered to develop the engineers and scientists who can manage problems in construction and operation of complex systems such as energy production systems, chemical plants, food processing systems, biological processes or problems in environmental conservation.

Duration:
4

Fees:
40000


3. Program: Bachelor of Mechanical Precision Engineering with Honours
Introduction:
Mechanical Precision Engineering program was designed for students who wish to aquire a s

In [21]:
#version 4: with other table
import pyodbc
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# SQL Query with Joins
query = """
SELECT 
    p.programName, 
    p.introduction, 
    p.duration, 
    p.fees, 
    u.uniNameEng, 
    b.location,
    b.address
FROM 
    Programme p
JOIN 
    University u ON p.uniID = u.uniID
JOIN 
    Branch b ON p.branchID = b.branchID
"""

# Execute the query
cursor.execute(query)
rows = cursor.fetchall()

# Get column names
columns = [column[0] for column in cursor.description]

# Convert data to a DataFrame
df = pd.DataFrame.from_records(rows, columns=columns)

# Close the connection
cursor.close()
conn.close()

# Ensure all relevant columns are strings and handle missing values
df['programName'] = df['programName'].astype(str).fillna('')
df['introduction'] = df['introduction'].astype(str).fillna('')
df['duration'] = df['duration'].astype(str).fillna('')
df['fees'] = pd.to_numeric(df['fees'], errors='coerce').fillna(0)
df['uniNameEng'] = df['uniNameEng'].astype(str).fillna('')
df['address'] = df['address'].astype(str).fillna('')

# Combine text fields into a single column for TF-IDF
df['combined'] = df['programName'] + ' ' + df['introduction'] + ' ' + df['duration'] + ' ' + df['uniNameEng'] + ' ' + df['address']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined'])

print("EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?")
print()

def calculate_similarity(query_vec, tfidf_matrix):
    return cosine_similarity(query_vec, tfidf_matrix)

while True:
    # Get user input
    user_input = input("> What is your programme criteria? ")
    
    # Exit loop if user enters 'exit'
    if user_input.lower() == 'exit':
        print("EduBot: I am happy to be at your service. See you again!")
        break
    
    # Generate the user input vector using the TF-IDF model
    query_vec = vectorizer.transform([user_input])
    
    # Calculate similarity between user input vector and database vectors
    similarities = calculate_similarity(query_vec, tfidf_matrix)

    # Get the indices of programmes sorted by similarity (in descending order)
    top_indices = np.argsort(similarities[0])[::-1]
    
    # Get top 10 results to apply numerical filtering
    top_results = df.iloc[top_indices[:10]]

    if top_results.empty:
        print("EduBot: No programs found based on the text criteria.")
        print("Please adjust your criteria or try a different query.")
        continue
    
    # Example numerical filters (modify as needed)
    max_fees = float(input("> Enter maximum fees: "))
    max_duration = int(input("> Enter maximum duration in years: "))

    # Apply numerical filters
    filtered_results = top_results[(top_results['fees'] <= max_fees) & (top_results['duration'].astype(int) <= max_duration)]
    
    if filtered_results.empty:
        print("EduBot: No programs found matching your numerical criteria.")
        print("Try adjusting the maximum fees and duration.")
        continue
    
    # Display the top 5 filtered results
    print("Top 5 Programme Recommendations:")
    print()
    
    for counter, index in enumerate(filtered_results.head(5).index, start=1):
        print()
        print(f"{counter}. Program: {df['programName'][index]}")
        print("Introduction:")
        print(df['introduction'][index])
        print()
        print("University:")
        print(df['uniNameEng'][index])
        print()
        print("Branch:")
        print(df['location'][index])
        print()
        print("Duration:")
        print(df['duration'][index])
        print()
        print("Fees:")
        print(df['fees'][index])
        print()
        
    print("Enter another set of programme criteria to generate other results, or type 'exit' to end the session...")


EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?

> What is your programme criteria? social economy
> Enter maximum fees: 40000
> Enter maximum duration in years: 4
Top 5 Programme Recommendations:


1. Program: Social Work - Bachelor (Hons)
Introduction:
Professional social work practice in Malaysia is becoming much more relevant in the last few decades because of rapid urbanization, changes in family structures, diminishing traditional support network and the increasing number of social problems. The main purpose of social work is about the enhancement of human well-being and the promotion of social justice and social development. The courses offered by this?programme?are intended to prepare and equip the students with knowledge on the interaction between human and society, Students learn theories and skills in social work to enable them to play their roles effectively in helping or working with individuals, groups and communi

> What is your programme criteria? exit
EduBot: I am happy to be at your service. See you again!


In [23]:
#version 5: cleaned data
import pyodbc
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# SQL Query with Joins
query = """
SELECT 
    p.programName, 
    p.introduction, 
    p.duration, 
    p.fees, 
    u.uniNameEng, 
    b.location,
    b.address
FROM 
    Programme p
JOIN 
    University u ON p.uniID = u.uniID
JOIN 
    Branch b ON p.branchID = b.branchID
"""

# Execute the query
cursor.execute(query)
rows = cursor.fetchall()

# Get column names
columns = [column[0] for column in cursor.description]

# Convert data to a DataFrame
df = pd.DataFrame.from_records(rows, columns=columns)

# Close the connection
cursor.close()
conn.close()

# Define data cleaning functions
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_specific_characters(text, char_list):
    for char in char_list:
        text = text.replace(char, '')
    return text

# Clean the DataFrame columns
df['introduction'] = df['introduction'].astype(str).fillna('').apply(remove_html_tags)
df['introduction'] = df['introduction'].apply(lambda x: remove_specific_characters(x, ['?']))

# Ensure all relevant columns are strings and handle missing values
df['programName'] = df['programName'].astype(str).fillna('')
df['duration'] = df['duration'].astype(str).fillna('')
df['fees'] = pd.to_numeric(df['fees'], errors='coerce').fillna(0)
df['uniNameEng'] = df['uniNameEng'].astype(str).fillna('')
df['address'] = df['address'].astype(str).fillna('')

# Combine text fields into a single column for TF-IDF
df['combined'] = df['programName'] + ' ' + df['introduction'] + ' ' + df['duration'] + ' ' + df['uniNameEng'] + ' ' + df['address']

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df['combined'])

print("EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?")
print()

def calculate_similarity(query_vec, tfidf_matrix):
    return cosine_similarity(query_vec, tfidf_matrix)

while True:
    # Get user input
    user_input = input("> What is your programme criteria? ")
    
    # Exit loop if user enters 'exit'
    if user_input.lower() == 'exit':
        print("EduBot: I am happy to be at your service. See you again!")
        break
    
    # Generate the user input vector using the TF-IDF model
    query_vec = vectorizer.transform([user_input])
    
    # Calculate similarity between user input vector and database vectors
    similarities = calculate_similarity(query_vec, tfidf_matrix)

    # Get the indices of programmes sorted by similarity (in descending order)
    top_indices = np.argsort(similarities[0])[::-1]
    
    # Get top 10 results to apply numerical filtering
    top_results = df.iloc[top_indices[:10]]

    if top_results.empty:
        print("EduBot: No programs found based on the text criteria.")
        print("Please adjust your criteria or try a different query.")
        continue
    
    # Example numerical filters (modify as needed)
    max_fees = float(input("> Enter maximum fees: "))
    max_duration = int(input("> Enter maximum duration in years: "))

    # Apply numerical filters
    filtered_results = top_results[(top_results['fees'] <= max_fees) & (top_results['duration'].astype(int) <= max_duration)]
    
    if filtered_results.empty:
        print("EduBot: No programs found matching your numerical criteria.")
        print("Try adjusting the maximum fees and duration.")
        continue
    
    # Display the top 5 filtered results
    print("Top 5 Programme Recommendations:")
    print()
    
    for counter, index in enumerate(filtered_results.head(5).index, start=1):
        print()
        print(f"{counter}. Program: {df['programName'][index]}")
        print("Introduction:")
        print(df['introduction'][index])
        print()
        print("University:")
        print(df['uniNameEng'][index])
        print()
        print("Branch:")
        print(df['location'][index])
        print()
        print("Duration:")
        print(df['duration'][index])
        print()
        print("Fees:")
        print(df['fees'][index])
        print()
        
    print("Enter another set of programme criteria to generate other results, or type 'exit' to end the session...")


EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?

> What is your programme criteria? computer science
> Enter maximum fees: 40000
> Enter maximum duration in years: 4
Top 5 Programme Recommendations:


1. Program: Bachelor Of Computer Engineering Technology (Computer Systems) With Honours

Introduction:
With the rapid development in the area of electronic and computer engineering, computer based equipments not only being used industries but has become a part of our daily lives.  Computer based equipments has been used and implemented for various purposes. Computer based machinery, water irrigation system, smart home system, mobile phone; tablet PC and gaming console are the examples of the products that require knowledge in computer system engineering. Bachelor of Computer Engineering Technology (Computer System) with Honours integrates electronic engineering and computer science knowledge together. Student will be able to learn

In [24]:
pip install gensim


  Obtaining dependency information for FuzzyTM>=0.4.0 from https://files.pythonhosted.org/packages/2d/30/074bac7a25866a2807c1005c7852c0139ac22ba837871fc01f16df29b9dc/FuzzyTM-2.0.9-py3-none-any.whl.metadata
  Obtaining dependency information for pyfume from https://files.pythonhosted.org/packages/ed/ea/a3b120e251145dcdb10777f2bc5f18b1496fd999d705a178c1b0ad947ce1/pyFUME-0.3.4-py3-none-any.whl.metadata
  Obtaining dependency information for scipy>=1.7.0 from https://files.pythonhosted.org/packages/65/76/903324159e4a3566e518c558aeb21571d642f781d842d8dd0fd9c6b0645a/scipy-1.10.1-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     -------------------- ------------------- 30.7/59.0 kB ? eta -:--:--
     ---------------------------------------- 59.0/59.0 kB 1.6 MB/s eta 0:00:00
  Obtaining dependency information for numpy>=1.18.5 from https://files.pythonhosted.org/packages/d8/ec/ebef2f7d7c28503f958f0f8b992e7ce606fb74f9e891199329d5f5f8

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\Software\\Anaconda\\Lib\\site-packages\\~umpy\\core\\_multiarray_tests.cp311-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [28]:
from gensim.models import Word2Vec

# Example Word2Vec model
model = Word2Vec(sentences=[['example', 'sentence']], vector_size=100, window=5, min_count=1, sg=0)

# Save the model
model.save('D:\Software\Anaconda\model.bin')


In [30]:
from gensim.models import KeyedVectors

# Load a pre-trained Word2Vec model (you need to download the model beforehand)
model_path = 'D:\Software\Anaconda\model.bin'  # Update this path to the location of your Word2Vec model
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [33]:
from gensim.models import KeyedVectors

# Use raw string to avoid escaping issues
model_path = r'D:\Software\Anaconda\model.bin'

# Attempt to load the model
try:
    # Adjust binary parameter based on file format
    word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)
    print("Model loaded successfully.")
except UnicodeDecodeError as e:
    print(f"Failed to load model: {e}")
except Exception as e:
    print(f"An error occurred: {e}")


Failed to load model: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte


In [4]:
import pyodbc

# List all available ODBC drivers
print(pyodbc.drivers())


['SQL Server', 'Oracle in XE', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 18 for SQL Server']


In [34]:
#enquiry chatbot
import re

# Define a dictionary of rules
rules = {
    'university': 'You can find information about universities on our website, including campus details and admission procedures.',
    'college': 'We provide details about various colleges including their rankings and facilities.',
    'program': 'We offer a range of programs including undergraduate, graduate, and diploma courses.',
    'course': 'Find information about courses, including course content and career prospects.',
    'degree': 'Explore our degree programs and their details on our website.',
    'major': 'We have a variety of majors available in different fields.',
    'fees': 'Information about fees for various programs is available on our fees page.',
    'duration': 'The duration of programs varies depending on the type of program. Please check the specific program details.',
    'admission': 'Check our admission criteria and process on the admissions page.',
    'requirements': 'Find out about the requirements for different programs on our requirements page.'
}

def classify_query(query):
    query = query.lower()  # Convert query to lowercase for uniformity

    for keyword, response in rules.items():
        if keyword in query:
            return response
    
    return "Sorry, I couldn't find the information you're looking for. Can you please provide more details?"

# Example user queries
user_queries = [
    "Tell me about the university fees.",
    "What are the requirements for the engineering program?",
    "How long is the course duration?",
    "Can you provide details about the university campus?"
]

# Process each query
for query in user_queries:
    response = classify_query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print()


Query: Tell me about the university fees.
Response: You can find information about universities on our website, including campus details and admission procedures.

Query: What are the requirements for the engineering program?
Response: We offer a range of programs including undergraduate, graduate, and diploma courses.

Query: How long is the course duration?
Response: Find information about courses, including course content and career prospects.

Query: Can you provide details about the university campus?
Response: You can find information about universities on our website, including campus details and admission procedures.



In [40]:
import pyodbc
import pandas as pd
import re

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

def fetch_program_details(program_name):
    query = """
    SELECT 
        p.programName, 
        p.introduction, 
        p.duration, 
        p.fees, 
        u.uniNameEng, 
        b.location
    FROM 
        Programme p
    JOIN 
        University u ON p.uniID = u.uniID
    JOIN 
        Branch b ON p.branchID = b.branchID
    WHERE 
        p.programName = ?
    """
    cursor.execute(query, (program_name,))
    result = cursor.fetchone()
    if result:
        return {
            'programName': result.programName,
            'introduction': result.introduction,
            'duration': result.duration,
            'fees': result.fees,
            'university': result.uniNameEng,
            'location': result.location
        }
    else:
        return None

def fetch_university_details(university_name):
    query = """
    SELECT 
        uniNameEng, 
        location, 
        address
    FROM 
        University
    WHERE 
        uniNameEng = ?
    """
    cursor.execute(query, (university_name,))
    result = cursor.fetchone()
    if result:
        return {
            'universityName': result.uniNameEng,
            'location': result.location,
            'address': result.address
        }
    else:
        return None

def classify_query(query):
    query = query.lower()  # Convert query to lowercase for uniformity

    if 'fees' in query:
        program_name = extract_program_name(query)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nFees: {details['fees']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the fee details."
    
    elif 'duration' in query:
        program_name = extract_program_name(query)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nDuration: {details['duration']}"
            else:
                return "No program found with the given name."
    
    elif 'university' in query:
        university_name = extract_university_name(query)
        if university_name:
            details = fetch_university_details(university_name)
            if details:
                return f"University: {details['universityName']}\nLocation: {details['location']}\nAddress: {details['address']}"
            else:
                return "No university found with the given name."
    
    else:
        return "Sorry, I couldn't understand your query. Please provide more details."

def extract_program_name(query):
    # Implement a method to extract program names from the query
    # This could be a simple heuristic or a more advanced NLP approach
    match = re.search(r'program\s+([a-zA-Z\s]+)', query)
    return match.group(1).strip() if match else None

def extract_university_name(query):
    # Implement a method to extract university names from the query
    # This could be a simple heuristic or a more advanced NLP approach
    match = re.search(r'university\s+([a-zA-Z\s]+)', query)
    return match.group(1).strip() if match else None

# Example user queries
user_queries = [
    "Tell me the fees for the BACHELOR OF SCIENCE IN BIOCHEMISTRY program.",
    "What is the duration of the Biochemistry - Bachelor (Hons) program?",
    "Give me details about Tunku Abdul Rahman University.",
    "How long is the duration for the Biochemistry - Bachelor (Hons) program?"
]

# Process each query
for query in user_queries:
    response = classify_query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print()

# Close the database connection
cursor.close()
conn.close()


Query: Tell me the fees for the BACHELOR OF SCIENCE IN BIOCHEMISTRY program.
Response: Please specify the program name to get the fee details.

Query: What is the duration of the Biochemistry - Bachelor (Hons) program?
Response: None

Query: Give me details about Tunku Abdul Rahman University.
Response: None

Query: How long is the duration for the Biochemistry - Bachelor (Hons) program?
Response: None



In [45]:
import pyodbc
import pandas as pd
import re

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

def fetch_program_details(program_name):
    query = """
    SELECT 
        p.programName, 
        p.introduction, 
        p.duration, 
        p.fees, 
        u.uniNameEng, 
        b.location
    FROM 
        Programme p
    JOIN 
        University u ON p.uniID = u.uniID
    JOIN 
        Branch b ON p.branchID = b.branchID
    WHERE 
        p.programName LIKE ?
    """
    cursor.execute(query, ('%' + program_name + '%',))
    result = cursor.fetchone()
    if result:
        return {
            'programName': result.programName,
            'introduction': result.introduction,
            'duration': result.duration,
            'fees': result.fees,
            'university': result.uniNameEng,
            'location': result.location
        }
    else:
        return None

def fetch_university_details(university_name):
    query = """
    SELECT 
        uniNameEng, 
        location, 
        address
    FROM 
        University
    WHERE 
        uniNameEng LIKE ?
    """
    cursor.execute(query, ('%' + university_name + '%',))
    result = cursor.fetchone()
    if result:
        return {
            'universityName': result.uniNameEng,
            'location': result.location,
            'address': result.address
        }
    else:
        return None

def classify_query(query):
    query = query.lower()  # Convert query to lowercase for uniformity

    if 'fees' in query:
        program_name = extract_program_name(query)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nFees: {details['fees']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the fee details."
    
    elif 'duration' in query:
        program_name = extract_program_name(query)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nDuration: {details['duration']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the duration details."
    
    elif 'university' in query:
        university_name = extract_university_name(query)
        if university_name:
            details = fetch_university_details(university_name)
            if details:
                return f"University: {details['universityName']}\nLocation: {details['location']}\nAddress: {details['address']}"
            else:
                return "No university found with the given name."
        else:
            return "Please specify the university name to get the details."
    
    else:
        return "Sorry, I couldn't understand your query. Please provide more details."

def extract_program_name(query):
    # Extract program names from the query using regex
    match = re.search(r'program\s+([\w\s-]+)', query, re.IGNORECASE)
    return match.group(1).strip() if match else None

def extract_university_name(query):
    # Extract university names from the query using regex
    match = re.search(r'university\s+([\w\s-]+)', query, re.IGNORECASE)
    return match.group(1).strip() if match else None

# Example user queries
user_queries = [
    "Tell me the fees for the BACHELOR OF SCIENCE IN BIOCHEMISTRY program.",
    "What is the duration of the Biochemistry - Bachelor (Hons) program?",
    "Give me details about Tunku Abdul Rahman University.",
    "How long is the duration for the Biochemistry - Bachelor (Hons) program?"
]

# Process each query
for query in user_queries:
    response = classify_query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print()

# Close the database connection
cursor.close()
conn.close()


Query: Tell me the fees for the BACHELOR OF SCIENCE IN BIOCHEMISTRY program.
Response: Please specify the program name to get the fee details.

Query: What is the duration of the Biochemistry - Bachelor (Hons) program?
Response: Please specify the program name to get the duration details.

Query: Give me details about Tunku Abdul Rahman University.
Response: Please specify the university name to get the details.

Query: How long is the duration for the Biochemistry - Bachelor (Hons) program?
Response: Please specify the program name to get the duration details.



In [61]:
import pyodbc
import pandas as pd
import re

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Fetch all program names and university names
def fetch_all_names():
    program_query = "SELECT programName FROM Programme"
    university_query = "SELECT uniNameEng FROM University"
    
    cursor.execute(program_query)
    program_names = [row.programName for row in cursor.fetchall()]
    
    cursor.execute(university_query)
    university_names = [row.uniNameEng for row in cursor.fetchall()]
    
    return program_names, university_names

program_names, university_names = fetch_all_names()

def fetch_program_details(program_name):
    query = """
    SELECT 
        p.programName, 
        p.introduction, 
        p.duration, 
        p.fees, 
        u.uniNameEng, 
        b.location
    FROM 
        Programme p
    JOIN 
        University u ON p.uniID = u.uniID
    JOIN 
        Branch b ON u.uniID = b.uniID  -- Corrected join condition
    WHERE 
        p.programName = ?
    """
    cursor.execute(query, (program_name,))
    result = cursor.fetchone()
    if result:
        return {
            'programName': result.programName,
            'introduction': result.introduction,
            'duration': result.duration,
            'fees': result.fees,
            'university': result.uniNameEng,
            'location': result.location
        }
    else:
        return None

def fetch_university_details(university_name):
    query = """
    SELECT 
        u.uniNameEng, 
        u.uniAcronym,
        u.foundationYear,
        u.uniType,
        b.location,
        b.address
    FROM 
        University u
    JOIN 
        Branch b ON u.uniID = b.uniID  -- Corrected join condition
    WHERE 
        u.uniNameEng = ?
    """
    cursor.execute(query, (university_name,))
    result = cursor.fetchone()
    if result:
        return {
            'universityName': result.uniNameEng,
            'universityAcronym': result.uniAcronym,
            'foundationYear': result.foundationYear,
            'uniType': result.uniType,
            'location': result.location,
            'address': result.address
        }
    else:
        return None

def match_name(query, names_list):
    query = query.lower()
    for name in names_list:
        if name.lower() in query:
            return name
    return None

def classify_query(query):
    query = query.lower()  # Convert query to lowercase for uniformity

    if 'fees' in query:
        program_name = match_name(query, program_names)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nFees: {details['fees']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the fee details."
    
    elif 'duration' in query:
        program_name = match_name(query, program_names)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nDuration: {details['duration']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the duration details."
    
    elif 'university' in query:
        university_name = match_name(query, university_names)
        if university_name:
            details = fetch_university_details(university_name)
            if details:
                return (f"University: {details['universityName']}\nAcronym: {details['universityAcronym']}\n"
                        f"Founded Year: {details['foundationYear']}\nUniversity Type: {details['uniType']}\n"
                        f"Location: {details['location']}\nAddress: {details['address']}")
            else:
                return "No university found with the given name."
        else:
            return "Please specify the university name to get the details."
    
    else:
        return "Sorry, I couldn't understand your query. Please provide more details."

# Example user queries
user_queries = [
    "Tell me the fees for the BACHELOR OF SCIENCE IN BIOCHEMISTRY program.",
    "What is the duration of the Biochemistry - Bachelor (Hons) program?",
    "Give me details about Sultan Zainal Abidin University.",
    "How long is the duration for the Biochemistry - Bachelor (Hons) program?"
]

# Process each query
for query in user_queries:
    response = classify_query(query)
    print(f"Query: {query}")
    print(f"Response: {response}")
    print()

# Close the database connection
cursor.close()
conn.close()


Query: Tell me the fees for the BACHELOR OF SCIENCE IN BIOCHEMISTRY program.
Response: Program: BACHELOR OF SCIENCE IN BIOCHEMISTRY
Fees: 11200

Query: What is the duration of the Biochemistry - Bachelor (Hons) program?
Response: Program: Biochemistry - Bachelor (Hons)
Duration: 4

Query: Give me details about Sultan Zainal Abidin University.
Response: University: Sultan Zainal Abidin University
Acronym: UniSZA
Founded Year: 1980 (as KUSZA)
University Type: Islamic university
Location: Kuala Terengganu, Terengganu
Address: Universiti Sultan Zainal Abidin, Kampung Gong Badak, 21300, Terengganu

Query: How long is the duration for the Biochemistry - Bachelor (Hons) program?
Response: Program: Biochemistry - Bachelor (Hons)
Duration: 4



In [1]:
#Final code
import pyodbc
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Fetch all program names and university names
def fetch_all_names():
    program_query = "SELECT programName FROM Programme"
    university_query = "SELECT uniNameEng FROM University"
    
    cursor.execute(program_query)
    program_names = [row.programName for row in cursor.fetchall()]
    
    cursor.execute(university_query)
    university_names = [row.uniNameEng for row in cursor.fetchall()]
    
    return program_names, university_names

program_names, university_names = fetch_all_names()

# Fetch program details
def fetch_program_details(program_name):
    query = """
    SELECT 
        p.programName, 
        p.introduction, 
        p.duration, 
        p.fees, 
        u.uniNameEng, 
        b.location
    FROM 
        Programme p
    JOIN 
        University u ON p.uniID = u.uniID
    JOIN 
        Branch b ON u.uniID = b.uniID
    WHERE 
        p.programName = ?
    """
    cursor.execute(query, (program_name,))
    result = cursor.fetchone()
    if result:
        return {
            'programName': result.programName,
            'introduction': result.introduction,
            'duration': result.duration,
            'fees': result.fees,
            'university': result.uniNameEng,
            'location': result.location
        }
    else:
        return None

# Fetch university details
def fetch_university_details(university_name):
    query = """
    SELECT 
        u.uniNameEng, 
        u.uniAcronym,
        u.foundationYear,
        u.uniType,
        b.location,
        b.address
    FROM 
        University u
    JOIN 
        Branch b ON u.uniID = b.uniID
    WHERE 
        u.uniNameEng = ?
    """
    cursor.execute(query, (university_name,))
    result = cursor.fetchone()
    if result:
        return {
            'universityName': result.uniNameEng,
            'universityAcronym': result.uniAcronym,
            'foundationYear': result.foundationYear,
            'uniType': result.uniType,
            'location': result.location,
            'address': result.address
        }
    else:
        return None

# Match names from a list
def match_name(query, names_list):
    query = query.lower()
    for name in names_list:
        if name.lower() in query:
            return name
    return None

# Classify query to provide information on a university or program
def classify_query(query):
    query = query.lower()

    if 'fees' in query:
        program_name = match_name(query, program_names)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nFees: {details['fees']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the fee details."
    
    elif 'duration' in query:
        program_name = match_name(query, program_names)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nDuration: {details['duration']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the duration details."
    
    elif 'university' in query:
        university_name = match_name(query, university_names)
        if university_name:
            details = fetch_university_details(university_name)
            if details:
                return (f"University: {details['universityName']}\nAcronym: {details['universityAcronym']}\n"
                        f"Founded Year: {details['foundationYear']}\nUniversity Type: {details['uniType']}\n"
                        f"Location: {details['location']}\nAddress: {details['address']}")
            else:
                return "No university found with the given name."
        else:
            return "Please specify the university name to get the details."
    
    else:
        return "Sorry, I couldn't understand your query. Please provide more details."

# EduBot for recommending programs based on criteria
def university_recommendation():
    # SQL Query with Joins
    query = """
    SELECT 
        p.programName, 
        p.introduction, 
        p.duration, 
        p.fees, 
        u.uniNameEng, 
        b.location,
        b.address
    FROM 
        Programme p
    JOIN 
        University u ON p.uniID = u.uniID
    JOIN 
        Branch b ON p.branchID = b.branchID
    """
    
    # Execute the query
    cursor.execute(query)
    rows = cursor.fetchall()
    
    # Get column names
    columns = [column[0] for column in cursor.description]
    
    # Convert data to a DataFrame
    df = pd.DataFrame.from_records(rows, columns=columns)

    # Define data cleaning functions
    def remove_html_tags(text):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)

    def remove_specific_characters(text, char_list):
        for char in char_list:
            text = text.replace(char, '')
        return text

    # Clean the DataFrame columns
    df['introduction'] = df['introduction'].astype(str).fillna('').apply(remove_html_tags)
    df['introduction'] = df['introduction'].apply(lambda x: remove_specific_characters(x, ['?']))

    # Ensure all relevant columns are strings and handle missing values
    df['programName'] = df['programName'].astype(str).fillna('')
    df['duration'] = df['duration'].astype(str).fillna('')
    df['fees'] = pd.to_numeric(df['fees'], errors='coerce').fillna(0)
    df['uniNameEng'] = df['uniNameEng'].astype(str).fillna('')
    df['address'] = df['address'].astype(str).fillna('')

    # Combine text fields into a single column for TF-IDF
    df['combined'] = df['programName'] + ' ' + df['introduction'] + ' ' + df['duration'] + ' ' + df['uniNameEng'] + ' ' + df['address']

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['combined'])

    print("EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?")
    print()

    def calculate_similarity(query_vec, tfidf_matrix):
        return cosine_similarity(query_vec, tfidf_matrix)

    while True:
        # Get user input
        user_input = input("> What is your programme criteria? ")
        
        # Exit loop if user enters 'exit'
        if user_input.lower() == 'exit':
            print("EduBot: I am happy to be at your service. See you again!")
            break
        
        # Generate the user input vector using the TF-IDF model
        query_vec = vectorizer.transform([user_input])
        
        # Calculate similarity between user input vector and database vectors
        similarities = calculate_similarity(query_vec, tfidf_matrix)

        # Get the indices of programmes sorted by similarity (in descending order)
        top_indices = np.argsort(similarities[0])[::-1]
        
        # Get top 10 results to apply numerical filtering
        top_results = df.iloc[top_indices[:10]]

        if top_results.empty:
            print("EduBot: No programs found based on the text criteria.")
            print("Please adjust your criteria or try a different query.")
            continue
        
        # Example numerical filters (modify as needed)
        max_fees = float(input("> Enter maximum fees: "))
        max_duration = int(input("> Enter maximum duration in years: "))

        # Apply numerical filters
        filtered_results = top_results[(top_results['fees'] <= max_fees) & (top_results['duration'].astype(int) <= max_duration)]
        
        if filtered_results.empty:
            print("EduBot: No programs found matching your numerical criteria.")
            print("Try adjusting the maximum fees and duration.")
            continue
        
        # Display the top 5 filtered results
        print("Top 5 Programme Recommendations:")
        print()
        
        for counter, index in enumerate(filtered_results.head(5).index, start=1):
            print()
            print(f"{counter}. Program: {df['programName'][index]}")
            print("Introduction:")
            print(df['introduction'][index])
            print()
            print("University:")
            print(df['uniNameEng'][index])
            print()
            print("Branch:")
            print(df['location'][index])
            print()
            print("Duration:")
            print(df['duration'][index])
            print()
            print("Fees:")
            print(df['fees'][index])
            print()
            
        print("Enter another set of programme criteria to generate other results, or type 'exit' to end the session...")

# Main function to choose between the algorithms
def main():
    print("EduBot: Hello, I am EduBot. What would you like me to help today?")
    print("EduBot: Choose an option:")
    print("1. Get a recommendation for a university based on criteria")
    print("2. Provide more information on a university or a program")
    
    choice = input("Enter the number of your choice: ").strip()
    
    if choice == '1':
        university_recommendation()
    elif choice == '2':
        print("EduBot: Hello, I am EduBot. Ask me about universities or programs.")
        print()
        while True:
            query = input("> ")
            if query.lower() == 'exit':
                print("EduBot: I am happy to be at your service. See you again!")
                break
            response = classify_query(query)
            print(response)
            print()
    else:
        print("Invalid choice. Please enter 1 or 2.")

if __name__ == "__main__":
    main()

# Close the database connection
cursor.close()
conn.close()


Choose an option:
1. Get a recommendation for a university based on criteria
2. Provide more information on a university or a program
Enter the number of your choice: 1
EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?

> What is your programme criteria? kuala lumpur computer science
> Enter maximum fees: 30000
> Enter maximum duration in years: 4
Top 5 Programme Recommendations:


1. Program: BACHELOR OF COMPUTER SCIENCE (COMPUTER SYSTEM AND NETWORK)
Introduction:
The purpose of this programme is to produce graduates who understand and able to apply the knowledge in networking and the theory of computer systems. The graduates will have the ability to apply critical thinking skills and advanced scientific techniques in solving computer-based problems in line with the rapidly changing computer systems and technology.

University:
University of Malaya

Branch:
Kuala Lumpur

Duration:
4

Fees:
9800


2. Program: Bachelor Of Computer

ValueError: could not convert string to float: '.'

In [ ]:
import pyodbc
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Set up the connection string
connection_string = (
    r'DRIVER={ODBC Driver 18 for SQL Server};'
    r'SERVER=(localdb)\MSSQLLocalDB;'
    r'AttachDbFilename=E:\UniFinder\App_Data\EduPathDatabase.mdf;'
    r'Trusted_Connection=yes;'
)

# Connect to the database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Fetch all program names and university names
def fetch_all_names():
    program_query = "SELECT programName FROM Programme"
    university_query = "SELECT uniNameEng FROM University"
    
    cursor.execute(program_query)
    program_names = [row.programName for row in cursor.fetchall()]
    
    cursor.execute(university_query)
    university_names = [row.uniNameEng for row in cursor.fetchall()]
    
    return program_names, university_names

program_names, university_names = fetch_all_names()

# Fetch program details
def fetch_program_details(program_name):
    query = """
    SELECT 
        p.programName, 
        p.introduction, 
        p.duration, 
        p.fees, 
        u.uniNameEng, 
        b.location
    FROM 
        Programme p
    JOIN 
        University u ON p.uniID = u.uniID
    JOIN 
        Branch b ON u.uniID = b.uniID
    WHERE 
        p.programName = ?
    """
    cursor.execute(query, (program_name,))
    result = cursor.fetchone()
    if result:
        return {
            'programName': result.programName,
            'introduction': result.introduction,
            'duration': result.duration,
            'fees': result.fees,
            'university': result.uniNameEng,
            'location': result.location
        }
    else:
        return None

# Fetch university details
def fetch_university_details(university_name):
    query = """
    SELECT 
        u.uniNameEng, 
        u.uniAcronym,
        u.foundationYear,
        u.uniType,
        b.location,
        b.address
    FROM 
        University u
    JOIN 
        Branch b ON u.uniID = b.uniID
    WHERE 
        u.uniNameEng = ?
    """
    cursor.execute(query, (university_name,))
    result = cursor.fetchone()
    if result:
        return {
            'universityName': result.uniNameEng,
            'universityAcronym': result.uniAcronym,
            'foundationYear': result.foundationYear,
            'uniType': result.uniType,
            'location': result.location,
            'address': result.address
        }
    else:
        return None

# Match names from a list
def match_name(query, names_list):
    query = query.lower()
    for name in names_list:
        if name.lower() in query:
            return name
    return None

# Classify query to provide information on a university or program
def classify_query(query):
    query = query.lower()

    if 'fees' in query:
        program_name = match_name(query, program_names)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nFees: {details['fees']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the fee details."
    
    elif 'duration' in query:
        program_name = match_name(query, program_names)
        if program_name:
            details = fetch_program_details(program_name)
            if details:
                return f"Program: {details['programName']}\nDuration: {details['duration']}"
            else:
                return "No program found with the given name."
        else:
            return "Please specify the program name to get the duration details."
    
    elif 'university' in query:
        university_name = match_name(query, university_names)
        if university_name:
            details = fetch_university_details(university_name)
            if details:
                return (f"University: {details['universityName']}\nAcronym: {details['universityAcronym']}\n"
                        f"Founded Year: {details['foundationYear']}\nUniversity Type: {details['uniType']}\n"
                        f"Location: {details['location']}\nAddress: {details['address']}")
            else:
                return "No university found with the given name."
        else:
            return "Please specify the university name to get the details."
    
    else:
        return "Sorry, I couldn't understand your query. Please provide more details."

# EduBot for recommending programs based on criteria
def university_recommendation():
    # SQL Query with Joins
    query = """
    SELECT 
        p.programName, 
        p.introduction, 
        p.duration, 
        p.fees, 
        u.uniNameEng, 
        b.location,
        b.address
    FROM 
        Programme p
    JOIN 
        University u ON p.uniID = u.uniID
    JOIN 
        Branch b ON p.branchID = b.branchID
    """
    
    # Execute the query
    cursor.execute(query)
    rows = cursor.fetchall()
    
    # Get column names
    columns = [column[0] for column in cursor.description]
    
    # Convert data to a DataFrame
    df = pd.DataFrame.from_records(rows, columns=columns)

    # Define data cleaning functions
    def remove_html_tags(text):
        clean = re.compile('<.*?>')
        return re.sub(clean, '', text)

    def remove_specific_characters(text, char_list):
        for char in char_list:
            text = text.replace(char, '')
        return text

    # Clean the DataFrame columns
    df['introduction'] = df['introduction'].astype(str).fillna('').apply(remove_html_tags)
    df['introduction'] = df['introduction'].apply(lambda x: remove_specific_characters(x, ['?']))

    # Ensure all relevant columns are strings and handle missing values
    df['programName'] = df['programName'].astype(str).fillna('')
    df['duration'] = df['duration'].astype(str).fillna('')
    df['fees'] = pd.to_numeric(df['fees'], errors='coerce').fillna(0)
    df['uniNameEng'] = df['uniNameEng'].astype(str).fillna('')
    df['address'] = df['address'].astype(str).fillna('')

    # Combine text fields into a single column for TF-IDF
    df['combined'] = df['programName'] + ' ' + df['introduction'] + ' ' + df['duration'] + ' ' + df['uniNameEng'] + ' ' + df['address']

    # Initialize TF-IDF Vectorizer
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['combined'])

    print("EduBot: Hello, I am EduBot, your program recommender assistant. What programme criteria do you have today?")
    print()

    def calculate_similarity(query_vec, tfidf_matrix):
        return cosine_similarity(query_vec, tfidf_matrix)

    while True:
        # Get user input
        user_input = input("> What is your programme criteria? ")
        
        # Exit loop if user enters 'exit'
        if user_input.lower() == 'exit':
            print("EduBot: I am happy to be at your service. See you again!")
            break
        
        # Generate the user input vector using the TF-IDF model
        query_vec = vectorizer.transform([user_input])
        
        # Calculate similarity between user input vector and database vectors
        similarities = calculate_similarity(query_vec, tfidf_matrix)

        # Get the indices of programmes sorted by similarity (in descending order)
        top_indices = np.argsort(similarities[0])[::-1]
        
        # Get top 10 results to apply numerical filtering
        top_results = df.iloc[top_indices[:10]]

        if top_results.empty:
            print("EduBot: No programs found based on the text criteria.")
            print("Please adjust your criteria or try a different query.")
            continue
        
        # Example numerical filters (modify as needed)
        max_fees = float(input("> Enter maximum fees: "))
        max_duration = int(input("> Enter maximum duration in years: "))

        # Apply numerical filters
        filtered_results = top_results[(top_results['fees'] <= max_fees) & (top_results['duration'].astype(int) <= max_duration)]
        
        if filtered_results.empty:
            print("EduBot: No programs found matching your numerical criteria.")
            print("Try adjusting the maximum fees and duration.")
            continue
        
        # Display the top 5 filtered results
        print("Top 5 Programme Recommendations:")
        print()
        
        for counter, index in enumerate(filtered_results.head(5).index, start=1):
            print()
            print(f"{counter}. Program: {df['programName'][index]}")
            print("Introduction:")
            print(df['introduction'][index])
            print()
            print("University:")
            print(df['uniNameEng'][index])
            print()
            print("Branch:")
            print(df['location'][index])
            print()
            print("Duration:")
            print(df['duration'][index])
            print()
            print("Fees:")
            print(df['fees'][index])
            print()
            
        print("Enter another set of programme criteria to generate other results, or type 'exit' to end the session...")

# Main function to choose between the algorithms
def main():
    print("Choose an option:")
    print("1. Get a recommendation for a university based on criteria")
    print("2. Provide more information on a university or a program")
    
    choice = input("Enter the number of your choice: ").strip()
    
    if choice == '1':
        university_recommendation()
    elif choice == '2':
        print("EduBot: Hello, I am EduBot. Ask me about universities or programs.")
        print()
        while True:
            query = input("> ")
            if query.lower() == 'exit':
                print("EduBot: I am happy to be at your service. See you again!")
                break
            response = classify_query(query)
            print(response)
            print()
    else:
        print("Invalid choice. Please enter 1 or 2.")

if __name__ == "__main__":
    main()

# Close the database connection
cursor.close()
conn.close()


Choose an option:
1. Get a recommendation for a university based on criteria
2. Provide more information on a university or a program
Enter the number of your choice: 2
EduBot: Hello, I am EduBot. Ask me about universities or programs.

> BACHELOR OF BIOMEDICAL ENGINEERING fees
Program: BACHELOR OF BIOMEDICAL ENGINEERING
Fees: 8800

> Tunku Abdul Rahman University of Management and Technology introduction
University: Tunku Abdul Rahman University of Management and Technology
Acronym: TARUMT
Founded Year: 1969
University Type: Private university
Location: Setapak
Address: null



Tunku Abdul Rahman University of Management and Technology
BACHELOR OF BIOMEDICAL ENGINEERING

In [5]:
pip install flask


Note: you may need to restart the kernel to use updated packages.
